# <b> Naive Bayes </b>
___

<b> Table of Content: </b>
<br> [Pipeline_1](#100)
<br> [Pipeline 2](#200)
<br> [Pipeline 3](#300)

Loading Modules and Datasets

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# import necessary packages  
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix, classification_report 
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss 

/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in

In [2]:
# read csv file to a pandas dataframe
df_pipeline1 = pd.read_csv("pipeline_1.csv")

<a id = "100"> <h2> Pipeline 1 </h2> </a>
___

> Declare Features and Target

In [3]:
# show all columns in dataset
list(df_pipeline1.columns)[:]

['Q4',
 'VisitorType_New_Visitor',
 'Q3',
 'TrafficType_2',
 'TrafficType_8',
 'TrafficType_3',
 'PageValues_iqr_yj_zscore',
 'Q1',
 'TrafficType_13',
 'ExitRates_iqr_yj_zscore',
 'OperatingSystems_3',
 'Administrative_Duration_iqr_yj_zscore',
 'TrafficType_1',
 'SpecialDay_0.8',
 'Month_Feb',
 'Browser_6',
 'SpecialDay_0.4',
 'TrafficType_20',
 'Informational_Duration_pp_iqr_yj_zscore',
 'Browser_12',
 'OperatingSystems_7',
 'TrafficType_16',
 'Revenue']

In [4]:
# Define Features and Target variables
X = df_pipeline1.iloc[:, :-1] # Features is all columns in the dataframe except the last column
Y = df_pipeline1.iloc[:, -1] # Target is the last column in the dataframe: 'Revenue'

In [5]:
# Split dataset into training set and test set 
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=2019) 

# Create a Gaussian Classifier
gnb = GaussianNB()

# Train the model using the training sets
gnb.fit(X_train, y_train)

print('Before OverSampling, the shape of train_X: {}'.format(X_train.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train.shape)) 
  
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '0': {}".format(sum(y_train == 0)))

# Predict the response for test dataset
y_pred = gnb.predict(X_test)

# print classification report 
print(classification_report(y_test, y_pred))

Before OverSampling, the shape of train_X: (8631, 22)
After OverSampling, the shape of train_y: (8631,) 

Before OverSampling, counts of label '1': 1340
Before OverSampling, counts of label '0': 7291
              precision    recall  f1-score   support

           0       0.96      0.76      0.85      3131
           1       0.38      0.84      0.53       568

    accuracy                           0.77      3699
   macro avg       0.67      0.80      0.69      3699
weighted avg       0.87      0.77      0.80      3699



In [6]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy Naive Bayes, pipeline 1:", metrics.accuracy_score(y_test, y_pred).round(4))

Accuracy Naive Bayes, pipeline 1: 0.7686


<b> 1.1 Synthetic Minority Oversampling Technique (SMOTE)

We oversample the dataset, because the classes within our target variable 'Revenue' are imbalanced:
* class 0: 84.53%
* class 1: 15.47%

Fore more information, click on detailed information from Prof. Jie Tao [link](https://github.com/DrJieTao/ba545-docs/blob/master/competition2/handling_imbalanced_data_part2.ipynb)

In [7]:
sm = SMOTE(random_state = 2019) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train) 

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

After OverSampling, the shape of train_X: (14582, 22)
After OverSampling, the shape of train_y: (14582,) 

After OverSampling, counts of label '1': 7291
After OverSampling, counts of label '0': 7291


/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


The SMOTE Algorithm has oversampled the instances in the minority class and made it equal to majority class:
* Both classes (0 & 1) now have 7291 instances, the dataset is balanced.
* Class 1 increased from 1340 instances to 7291 instances, an increase of 5951 instances of class 1.

In [8]:
gnb1 = GaussianNB()
gnb1.fit(X_train_res, y_train_res)
y_pred = gnb1.predict(X_test) 
  
# print classification report 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.49      0.65      3131
           1       0.25      0.93      0.39       568

    accuracy                           0.56      3699
   macro avg       0.61      0.71      0.52      3699
weighted avg       0.86      0.56      0.61      3699



In [9]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy Naive Bayes, pipeline 1:", metrics.accuracy_score(y_test, y_pred).round(4))

Accuracy Naive Bayes, pipeline 1: 0.5558


### Take-aways:
- The accuracy for predicting class 1 of our target (predicting if a web user will make a purchase is) is 39% after oversampling.
- Before oversampling the accuracy was 53%, which is a decrease of 14%. However after oversampling class 1, Our model is more unbiased.
- This is an increase in accuracy over random guessing class 1 correctly, from 15.47% to 39%, which is a 23.53% increase.

<b> 1.2 NearMiss Under-Sampling Technique

In [10]:
# apply near miss 
nr = NearMiss(random_state=123) 
  
X_train_miss, y_train_miss = nr.fit_sample(X_train, y_train) 
  
print('After Undersampling, the shape of train_X: {}'.format(X_train_miss.shape)) 
print('After Undersampling, the shape of train_y: {} \n'.format(y_train_miss.shape)) 
  
print("After Undersampling, counts of label '1': {}".format(sum(y_train_miss == 1))) 
print("After Undersampling, counts of label '0': {}".format(sum(y_train_miss == 0)))

After Undersampling, the shape of train_X: (2680, 22)
After Undersampling, the shape of train_y: (2680,) 

After Undersampling, counts of label '1': 1340
After Undersampling, counts of label '0': 1340


/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


The NearMiss Algorithm has undersampled the instances in the majority class and made it equal to minority class:
* Both classes (0 & 1) now have 1340 instances, the dataset is balanced.
* Class 0 decreased from 7291 instances to 1340 instances, a decrease of 5951 instances of class 0.

In [11]:
# train the model on train set 
gnb2 = GaussianNB()
gnb2.fit(X_train_miss, y_train_miss) 
y_pred = gnb2.predict(X_test) 

# print classification report 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      3131
           1       0.07      0.05      0.06       568

    accuracy                           0.75      3699
   macro avg       0.45      0.47      0.46      3699
weighted avg       0.72      0.75      0.73      3699



### Take-aways:
- The accuracy for predicting class 1 of our target (predicting if a web user will make a purchase is) is 9% after undersampling.
- Before undersampling it was 36.01%, which is a decrease of 27.01%. However after undersampling class 1, Our model is more unbiased.
- NearMiss performs worse with 9% than random guessing with 15.47%, which is a decrease of 6.47%.
- Oversampling with SMOTE however gives us a better F1 score and better accuracy which implies that SMOTE is probably a better technique to use for this dataset.
<br> (27.33% accuracy for SMOTE) > (9.00% accuracy for NearMiss) in predicting class 1.

<a id = "200"> <h2> Pipeline 2 </h2> </a>
___

<a id = "300"> <h2> Pipeline 3 </h2> </a>
___